In [117]:
import requests
from bs4 import BeautifulSoup as bs
import numpy as np
import matplotlib.pyplot as plt
import concurrent.futures
import pandas as pd
import re
from threading import Thread
import collections
import json
from datetime import datetime as dt
from fuzzywuzzy import fuzz
from countryinfo import CountryInfo
import pycountry

In [118]:
# the general idea is to predict the "happiness" of players in the team individually and then averaging
# the happiness and calling this number the measure of chemistry, simplistic but whatever.
# the happiness of one player will depend on if his team mates are of same or neighboring countries, we
# assume the player is being paid what he wants, and how many team mates are within 5 years of age. 


In [119]:
players_df = pd.read_csv('../data/processed.csv')
players_df.head()

,Name,Overall Rating,Potential,Value,Wage,Crossing,Finishing,Heading Accuracy,Short Passing,Volleys,...,GK Positioning,GK Reflexes,Positions,Nationality,Age,Height,Weight,Primary Position,Club,National Team
0,Matty Cash,78.0,83.0,20000000.0,50000.0,78.0,66.0,49.0,70.0,48.0,...,13.0,8.0,"['RB', 'RWB']",Poland,23.0,179.0,74.0,RB,Aston Villa,Poland
1,Gerónimo Rulli,81.0,81.0,17500000.0,33000.0,12.0,15.0,12.0,36.0,16.0,...,79.0,82.0,['GK'],Argentina,29.0,189.0,84.0,GK,Villarreal CF,No national team
2,Donyell Malen,79.0,85.0,28500000.0,56000.0,71.0,79.0,69.0,75.0,74.0,...,10.0,14.0,['ST'],Netherlands,22.0,179.0,78.0,ST,Borussia Dortmund,Netherlands
3,Oleksandr Zinchenko,79.0,82.0,22000000.0,90000.0,81.0,58.0,70.0,81.0,61.0,...,11.0,11.0,['LB'],Ukraine,24.0,175.0,64.0,LB,Manchester City,Ukraine
4,Rodrigo Hernández Cascante,87.0,89.0,90000000.0,190000.0,63.0,64.0,78.0,86.0,59.0,...,14.0,8.0,['CDM'],Spain,25.0,191.0,82.0,CDM,Manchester City,Spain


In [120]:
players_df['Primary Position'].unique()

array(['RB', 'GK', 'ST', 'LB', 'CDM', 'CB', 'CF', 'LW', 'CAM', 'CM', 'RM',
       'LM', 'RW', 'RWB', 'LWB'], dtype=object)

In [121]:
lang_dict = {}
for i in sorted(list(players_df['Nationality'].unique())):

    if i in ['China PR', 'Chinese Taipei']:
        country = CountryInfo('China')
        languages = country.languages()
        lang_dict[i] = languages

    elif i in ['Congo',  'Congo DR']:
        country = CountryInfo('Democratic Republic of the Congo')
        languages = country.languages()
        lang_dict[i] = languages

    elif i in ['England','Scotland', 'Wales','Northern Ireland']:
        country = CountryInfo('United Kingdom')
        languages = country.languages()
        lang_dict[i] = languages
    
    elif i ==  'Malaysia':
        country = CountryInfo('Singapore')
        languages = country.languages()
        lang_dict[i] = languages

    else:
    
        try:
            country = CountryInfo(i.title())
            languages = country.languages()
            lang_dict[i] = languages
        
        except:
            pass

In [122]:
data = [[i,str(lang_dict[i])] for i in lang_dict]
df = pd.DataFrame(data=data,columns=['Country','Languages'])

In [123]:
df.to_csv('../data/country_and_languages.csv', index=False)